In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [9]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Mato Grosso Do Sul - Produção de Cimento (t)', axis=1)
data

,Ano,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2008,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,0.808544,0.686768,0.746558,5.483335e+08,2.546644e+09,...,30.360011,952.583468,1388.122906,23.505311,16.064850,66.686091,3.455270,0.749040,8.011861,814.771000
1,2009,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,0.809433,0.689912,0.747506,5.302764e+08,2.679945e+09,...,31.066389,952.388003,1396.303897,23.414338,15.953541,66.658540,3.422201,0.748012,8.040524,693.327000
2,2010,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,0.809776,0.693020,0.748442,5.046425e+08,2.891930e+09,...,32.508565,952.362034,1428.326267,23.460429,15.531732,65.002039,3.316282,0.747440,8.003862,813.149000
3,2011,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,0.809583,0.696082,0.749364,4.854622e+08,3.091988e+09,...,34.904902,952.558014,1433.680808,23.635226,15.687149,64.896779,3.223002,0.747314,7.993144,923.248000
4,2012,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,0.808780,0.698913,0.750034,4.867661e+08,3.189506e+09,...,34.979340,952.759391,1421.572593,23.663605,15.955379,65.719662,3.178068,0.747266,6.360445,1008.624000
5,2013,5.452953e+07,2.278494e+06,16.940587,4.240902e+07,0.807595,0.702405,0.751453,5.225848e+08,3.093872e+09,...,33.811640,952.954665,1405.057523,23.757867,16.260264,66.470023,3.125032,0.748788,6.725372,1028.608000
6,2014,5.543864e+07,2.276794e+06,16.985287,4.261295e+07,0.805894,0.704772,0.752158,5.676413e+08,2.779340e+09,...,33.031265,953.098166,1382.443280,23.914606,16.608338,67.060540,3.088252,0.749938,7.077333,995.222143
7,2015,5.607449e+07,2.266815e+06,16.994005,4.270203e+07,0.803719,0.707587,0.752754,5.179392e+08,2.561731e+09,...,31.224126,953.111147,1377.219450,23.862880,16.783027,67.636795,3.072316,0.750599,7.357560,951.041314
8,2016,5.656732e+07,2.251674e+06,17.013733,4.279700e+07,0.809908,0.717698,0.756127,4.524144e+08,2.516859e+09,...,29.308459,952.984733,1353.708395,23.814967,16.659956,67.363296,3.049349,0.744326,7.332914,904.537543
9,2017,5.674497e+07,2.236955e+06,17.015525,4.281848e+07,0.806990,0.719489,0.755957,4.213689e+08,2.497710e+09,...,27.027249,953.036051,1260.888062,23.920271,16.556341,66.782526,2.972744,0.744629,7.012814,870.341000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
0,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,0.808544,0.686768,0.746558,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,30.360011,952.583468,1388.122906,23.505311,16.064850,66.686091,3.455270,0.749040,8.011861
1,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,0.809433,0.689912,0.747506,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,31.066389,952.388003,1396.303897,23.414338,15.953541,66.658540,3.422201,0.748012,8.040524
2,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,0.809776,0.693020,0.748442,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,32.508565,952.362034,1428.326267,23.460429,15.531732,65.002039,3.316282,0.747440,8.003862
3,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,0.809583,0.696082,0.749364,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,34.904902,952.558014,1433.680808,23.635226,15.687149,64.896779,3.223002,0.747314,7.993144
4,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,0.808780,0.698913,0.750034,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,34.979340,952.759391,1421.572593,23.663605,15.955379,65.719662,3.178068,0.747266,6.360445
5,5.452953e+07,2.278494e+06,16.940587,4.240902e+07,0.807595,0.702405,0.751453,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,33.811640,952.954665,1405.057523,23.757867,16.260264,66.470023,3.125032,0.748788,6.725372
6,5.543864e+07,2.276794e+06,16.985287,4.261295e+07,0.805894,0.704772,0.752158,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,33.031265,953.098166,1382.443280,23.914606,16.608338,67.060540,3.088252,0.749938,7.077333
7,5.607449e+07,2.266815e+06,16.994005,4.270203e+07,0.803719,0.707587,0.752754,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,31.224126,953.111147,1377.219450,23.862880,16.783027,67.636795,3.072316,0.750599,7.357560
8,5.656732e+07,2.251674e+06,17.013733,4.279700e+07,0.809908,0.717698,0.756127,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,29.308459,952.984733,1353.708395,23.814967,16.659956,67.363296,3.049349,0.744326,7.332914
9,5.674497e+07,2.236955e+06,17.015525,4.281848e+07,0.806990,0.719489,0.755957,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,27.027249,953.036051,1260.888062,23.920271,16.556341,66.782526,2.972744,0.744629,7.012814


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      693.327000
1      813.149000
2      923.248000
3     1008.624000
4     1028.608000
5      995.222143
6      951.041314
7      904.537543
8      870.341000
9      869.680000
10     902.144000
11    1038.045000
12    1092.056000
13    1152.060000
14            NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
0,-1.796747,-0.650239,-1.985110,-1.919451,0.518131,-1.517946,-1.602263,1.204752,-0.990234,0.166091,...,0.984445,-0.338768,-0.908635,0.325154,-1.091330,-0.334797,0.493297,1.511319,0.860252,1.264582
1,-1.624414,0.229010,-1.795763,-1.743254,0.799491,-1.254064,-1.303980,0.820657,-0.546688,0.428911,...,-0.051710,-0.064876,-1.590134,0.466286,-1.454581,-0.619536,0.469052,1.363350,0.357086,1.315878
2,-1.095430,1.202637,-0.988869,-1.025725,0.908067,-0.993168,-1.009842,0.275392,0.158672,0.847995,...,-0.115145,0.494314,-1.680674,1.018708,-1.270544,-1.698561,-0.988622,0.889403,0.077345,1.250267
3,-0.611803,1.422742,-0.324167,-0.455159,0.846962,-0.736133,-0.719926,-0.132596,0.824346,1.230291,...,0.633403,1.423471,-0.997382,1.111080,-0.572583,-1.300989,-1.081249,0.472017,0.015931,1.231086
4,-0.175626,1.137319,0.120809,0.000837,0.592735,-0.498516,-0.509304,-0.104860,1.148829,1.382743,...,-0.646476,1.452334,-0.295274,0.902200,-0.459269,-0.614833,-0.357135,0.270956,-0.007606,-1.690786
5,0.211695,0.659137,0.435031,0.370620,0.217752,-0.205323,-0.062964,0.657046,0.830615,1.112297,...,-0.757614,0.999569,0.385559,0.617296,-0.082883,0.165091,0.303163,0.033641,0.737208,-1.037715
6,0.539088,0.579898,0.701020,0.647859,-0.320327,-0.006617,0.158537,1.615453,-0.215960,0.282052,...,0.345772,0.696986,0.885881,0.227174,0.542969,1.055494,0.822801,-0.130934,1.299438,-0.407846
7,0.768073,0.114776,0.752894,0.768952,-1.008723,0.229633,0.346091,0.558230,-0.940033,-0.632425,...,1.298413,-0.003715,0.931139,0.137057,0.336428,1.502365,1.329890,-0.202240,1.623093,0.093648
8,0.945552,-0.590971,0.870290,0.898070,0.949764,1.078400,1.406722,-0.835563,-1.089341,-1.093812,...,1.596124,-0.746497,0.490392,-0.268535,0.145116,1.187538,1.089218,-0.305007,-1.446069,0.049540
9,1.009527,-1.277063,0.880952,0.927264,0.026583,1.228721,1.353185,-1.495939,-1.153056,-1.335633,...,-0.048438,-1.631015,0.669314,-1.869792,0.565589,0.922481,0.578156,-0.647783,-1.298100,-0.523309


In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      693.327000
1      813.149000
2      923.248000
3     1008.624000
4     1028.608000
5      995.222143
6      951.041314
7      904.537543
8      870.341000
9      869.680000
10     902.144000
11    1038.045000
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
12,0.585267,-1.32885,0.291975,0.434228,-2.576211,1.160996,0.103862,1.108405,2.635823,-0.189959,...,-2.260447,-0.26749,0.720445,0.131831,1.921027,-0.898095,-2.178339,-2.706799,-1.357455,-1.015947


In [15]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1092.056
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[166208090, 3066195302, 1233393896, 252257792, 1188438952, 4076685963, 3220677266, 387846092, 2165291755, 271748700, 1484453491, 2589474469, 813436540, 1815181361, 115694672, 2893345992, 357948324, 3882357760, 3674371888, 1390451558, 3724726079, 3558927274, 2222198994, 1563894198, 3452523583, 2601262608, 1849758917, 2564507151, 3994701978, 1580356327, 1497710344, 1096448540, 163377290, 4197461583, 1625644204, 854659208, 1024879368, 3782223246, 2744281138, 3894113844, 1430574429, 1388789713, 3051833965, 3494753782, 1914759936, 3318206080, 508378421, 242837532, 2156886403, 4095936979, 3848590128, 63712669, 392006110, 3332566627, 1118101657, 2317707238, 3990727738, 2701162542, 4110782572, 3810570052, 45834562, 2442348882, 106140509, 3820938995, 3250105014, 3710560750, 1943769362, 4169486442, 3576828475, 3974086181, 1485201356, 1073749078, 3702769946, 3614809946, 1214599251, 1887832306, 357871423, 1367149879, 3574392032, 625057611, 4282177355, 352962831, 1978242414, 3782379944, 1077914884,

val_loss: 5825.9453125


Step: 87 ___________________________________________
val_loss: 5752.28369140625


Step: 88 ___________________________________________
val_loss: 2909.530517578125


Step: 89 ___________________________________________
val_loss: 13180.3544921875


Step: 90 ___________________________________________
val_loss: 11098.3212890625


Step: 91 ___________________________________________
val_loss: 13177.748046875


Step: 92 ___________________________________________
val_loss: 11233.9814453125


Step: 93 ___________________________________________
val_loss: 10783.0556640625


Step: 94 ___________________________________________
val_loss: 10699.6953125


Step: 95 ___________________________________________
val_loss: 12307.982421875


Step: 96 ___________________________________________
val_loss: 10498.34375


Step: 97 ___________________________________________
val_loss: 12907.228515625


Step: 98 ___________________________________________
val_loss: 11737.998046875


St

In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 452ms/step - loss: 849392.6875 - val_loss: 1046259.8750
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 838394.2500 - val_loss: 404678.3125
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 1165267.7500 - val_loss: 749238.0625
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 960484.7500 - val_loss: 962883.1875
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 757643.1875 - val_loss: 1122872.8750
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 729653.6875 - val_loss: 1020541.4375
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 740342.4375 - val_loss: 1303539.5000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 708038.1875 - val_loss: 1010295.4375
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 678568.0625 - val_loss: 1050206.1250
Ep

1/1 [==============================] - 0s 13ms/step - loss: 195574.6719 - val_loss: 479836.4375
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 190645.1719 - val_loss: 465296.1875
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 188101.0781 - val_loss: 471900.5000
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 185469.7344 - val_loss: 457018.8438
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 182248.6875 - val_loss: 448657.2812
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 179936.4844 - val_loss: 448017.1250
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 176575.2031 - val_loss: 437577.4062
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 173833.9219 - val_loss: 435398.2188
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 171728.1719 - val_loss: 427573.0312
Epoch 83/10000
1

Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 61148.8594 - val_loss: 167499.2969
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 63537.3008 - val_loss: 174976.3594
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 62839.5469 - val_loss: 172497.6562
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 60080.5742 - val_loss: 172333.9375
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 61107.5898 - val_loss: 171179.7188
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 58050.7383 - val_loss: 163701.1406
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 66055.3203 - val_loss: 114443.6641
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 63402.0820 - val_loss: 160156.0469
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 55379.8789 - val_loss: 148496.4844
E

1/1 [==============================] - 0s 13ms/step - loss: 26430.3652 - val_loss: 63504.8008
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 23716.6328 - val_loss: 58487.2500
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 23182.3652 - val_loss: 56700.4336
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 26305.9160 - val_loss: 49672.1758
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 24073.8828 - val_loss: 61517.0781
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 23291.2598 - val_loss: 58841.1875
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 23370.6855 - val_loss: 58122.5781
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 22768.0566 - val_loss: 57885.5273
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 24276.6777 - val_loss: 51806.3047
Epoch 231/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 14418.0752 - val_loss: 28607.6211
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 14331.9648 - val_loss: 25953.3984
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 15015.5234 - val_loss: 29503.5762
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 14767.4688 - val_loss: 26768.6660
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 13682.5703 - val_loss: 34484.6523
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 11928.6123 - val_loss: 31999.4746
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 13191.0127 - val_loss: 33382.4375
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 14016.1338 - val_loss: 33079.7578
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 12067.6611 - val_loss: 26970.9453
Epoch 305/10000
1/1 [=====

...

1/1 [==============================] - 0s 16ms/step - loss: 8921.1250 - val_loss: 15181.2061
Epoch 1422/10000
1/1 [==============================] - 0s 16ms/step - loss: 8917.3115 - val_loss: 15128.8496
Epoch 1423/10000
1/1 [==============================] - 0s 16ms/step - loss: 9065.7119 - val_loss: 15012.7109
Epoch 1424/10000
1/1 [==============================] - 0s 16ms/step - loss: 9033.8955 - val_loss: 15014.6406
Epoch 1425/10000
1/1 [==============================] - 0s 16ms/step - loss: 8941.1846 - val_loss: 15333.5410
Epoch 1426/10000
1/1 [==============================] - 0s 16ms/step - loss: 8923.5957 - val_loss: 14917.1992
Epoch 1427/10000
1/1 [==============================] - 0s 16ms/step - loss: 8922.6162 - val_loss: 14974.4004
Epoch 1428/10000
1/1 [==============================] - 0s 16ms/step - loss: 8938.9893 - val_loss: 15372.9883
Epoch 1429/10000
1/1 [==============================] - 0s 15ms/step - loss: 8929.7900 - val_loss: 15240.3271
Epoch 1430/10000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 8912.2920 - val_loss: 14744.2051
Epoch 1496/10000
1/1 [==============================] - 0s 16ms/step - loss: 8941.6533 - val_loss: 15106.2764
Epoch 1497/10000
1/1 [==============================] - 0s 16ms/step - loss: 8912.4189 - val_loss: 14666.6230
Epoch 1498/10000
1/1 [==============================] - 0s 16ms/step - loss: 8907.3955 - val_loss: 14737.9658
Epoch 1499/10000
1/1 [==============================] - 0s 15ms/step - loss: 8984.1943 - val_loss: 13953.4150
Epoch 1500/10000
1/1 [==============================] - 0s 16ms/step - loss: 8918.1572 - val_loss: 14837.8984
Epoch 1501/10000
1/1 [==============================] - 0s 16ms/step - loss: 9370.9404 - val_loss: 15074.7861
Epoch 1502/10000
1/1 [==============================] - 0s 15ms/step - loss: 8921.0225 - val_loss: 14642.6543
Epoch 1503/10000
1/1 [==============================] - 0s 16ms/step - loss: 8937.4697 - val_loss: 14832.6348
Epoch 1504/10000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 8910.4453 - val_loss: 14376.0889
Epoch 1570/10000
1/1 [==============================] - 0s 16ms/step - loss: 8933.8135 - val_loss: 14791.5869
Epoch 1571/10000
1/1 [==============================] - 0s 16ms/step - loss: 8909.9658 - val_loss: 14385.6191
Epoch 1572/10000
1/1 [==============================] - 0s 16ms/step - loss: 8914.2168 - val_loss: 14123.8945
Epoch 1573/10000
1/1 [==============================] - 0s 16ms/step - loss: 8909.1943 - val_loss: 14487.8047
Epoch 1574/10000
1/1 [==============================] - 0s 16ms/step - loss: 8904.4082 - val_loss: 14434.4150
Epoch 1575/10000
1/1 [==============================] - 0s 16ms/step - loss: 9016.6787 - val_loss: 14858.5889
Epoch 1576/10000
1/1 [==============================] - 0s 17ms/step - loss: 8993.6006 - val_loss: 13647.8691
Epoch 1577/10000
1/1 [==============================] - 0s 16ms/step - loss: 8950.7217 - val_loss: 14886.3232
Epoch 1578/10000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 8978.8467 - val_loss: 14944.7041
Epoch 1644/10000
1/1 [==============================] - 0s 16ms/step - loss: 8909.4600 - val_loss: 14147.3340
Epoch 1645/10000
1/1 [==============================] - 0s 15ms/step - loss: 8902.3115 - val_loss: 14274.5615
Epoch 1646/10000
1/1 [==============================] - 0s 16ms/step - loss: 8909.6768 - val_loss: 14408.1602
Epoch 1647/10000
1/1 [==============================] - 0s 15ms/step - loss: 8904.0869 - val_loss: 14231.4678
Epoch 1648/10000
1/1 [==============================] - 0s 16ms/step - loss: 8916.0469 - val_loss: 14538.5547
Epoch 1649/10000
1/1 [==============================] - 0s 16ms/step - loss: 8903.5381 - val_loss: 14207.6387
Epoch 1650/10000
1/1 [==============================] - 0s 16ms/step - loss: 8903.4678 - val_loss: 14259.8203
Epoch 1651/10000
1/1 [==============================] - 0s 16ms/step - loss: 8976.1025 - val_loss: 14927.5479
Epoch 1652/10000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 8905.5508 - val_loss: 14107.4482
Epoch 1718/10000
1/1 [==============================] - 0s 16ms/step - loss: 8904.7275 - val_loss: 14102.9834
Epoch 1719/10000
1/1 [==============================] - 0s 16ms/step - loss: 8943.3164 - val_loss: 13636.3633
Epoch 1720/10000
1/1 [==============================] - 0s 16ms/step - loss: 8912.2148 - val_loss: 14185.8799
Epoch 1721/10000
1/1 [==============================] - 0s 16ms/step - loss: 8902.0166 - val_loss: 14186.5928
Epoch 1722/10000
1/1 [==============================] - 0s 16ms/step - loss: 8904.4443 - val_loss: 14081.3047
Epoch 1723/10000
1/1 [==============================] - 0s 16ms/step - loss: 8902.5479 - val_loss: 14097.8809
Epoch 1724/10000
1/1 [==============================] - 0s 16ms/step - loss: 8915.4541 - val_loss: 13849.1836
Epoch 1725/10000
1/1 [==============================] - 0s 16ms/step - loss: 8912.5986 - val_loss: 14021.6738
Epoch 1726/10000
1/1 [=====

In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 54ms/step
[1092.056](test_target) - [[1005.36383]](prediction) = 86.69216943359379


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
0,-0.975936,-1.203430,-0.914858,-0.922677,-1.361786,-1.227061,-1.227592,1.148205,-1.122043,-1.121418,...,1.412346,-1.064331,1.406054,-0.929666,1.219289,0.935499,0.724528,0.967433,1.323407,-0.437633
1,-0.398407,-0.041572,-0.476529,-0.466832,0.350506,0.004646,0.005713,0.140893,-0.184474,-0.185492,...,-0.643250,-0.274308,-0.571654,-0.458093,-1.230128,0.450745,0.689541,0.409623,-0.229909,1.383444
2,1.374343,1.245001,1.391386,1.389509,1.011279,1.222415,1.221878,-1.289098,1.306518,1.306910,...,-0.769096,1.338639,-0.834400,1.387758,0.010839,-1.386244,-1.414069,-1.377056,-1.093497,-0.945811


train_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
0,693.327
1,813.149
2,923.248


test_input:


...

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
12,0.585267,-1.32885,0.291975,0.434228,-2.576211,1.160996,0.103862,1.108405,2.635823,-0.189959,...,-2.260447,-0.26749,0.720445,0.131831,1.921027,-0.898095,-2.178339,-2.706799,-1.357455,-1.015947


test_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
12,1092.056


1/1 [==============================] - 0s 34ms/step
Error: 86.69216943359379




[923.24853515625,
 965.9329833984375,
 910.1416625976562,
 936.3448486328125,
 918.7501220703125,
 877.4513549804688,
 850.4639282226562,
 889.5369873046875,
 903.093017578125,
 1005.3638305664062]

In [24]:
display(targets)
display(predictions)

[1008.624,
 1028.608,
 995.2221428571428,
 951.0413142857142,
 904.5375428571429,
 870.341,
 869.68,
 902.144,
 1038.045,
 1092.056]

[923.24853515625,
 965.9329833984375,
 910.1416625976562,
 936.3448486328125,
 918.7501220703125,
 877.4513549804688,
 850.4639282226562,
 889.5369873046875,
 903.093017578125,
 1005.3638305664062]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

52.26175978794642

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.054099526099499